# Finetune ruGPT3Small on essays

## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

In [ ]:
!pip3 install transformers==2.8.0

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [120]:
data_path = "drive/My Drive/essays.txt"
!ls "$data_path"

'drive/My Drive/essays.txt'


## Prepare data

In [121]:
with open(data_path, "r") as file:
    text = file.read()

In [122]:
valid_size = 5

In [123]:
topics = []
all_essays = []
for line in text.split("</s>"):
    if "Тема:" in line and "Сочинение:" in line:
        essay_text = line.split("Сочинение:")
        if len(essay_text) == 2:
            topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = f"Сочинение: {essay_text}"
            essay_res = f"<s>{topic}\n{essay_text}</s>"
            all_essays.append(essay_res)
            topics.append(topic)

In [124]:
import numpy as np
import random

In [125]:
random.seed(1234)
np.random.seed(1234)

In [126]:
unique_topics = list(set(topics))

In [127]:
valid_topics = []

In [128]:
for _ in range(valid_size):
    # Use randint for more speed (on big lists it is faster)
    idx = np.random.randint(0, len(unique_topics))
    valid_topics.append(unique_topics[idx])

In [129]:
import nltk


train = []
valid = []
for topic, essay in zip(topics, all_essays):
    is_train = True
    for valid_topic in valid_topics:
        if (
            nltk.edit_distance(valid_topic, topic[:len(valid_topic)]) < 20 or
            nltk.edit_distance(valid_topic[:len(topic)], topic) < 20 or
            nltk.edit_distance(valid_topic[len(topic):], topic) < 20 or
            nltk.edit_distance(valid_topic, topic[len(valid_topic):]) < 20
            ):
            is_train = False
    if is_train:
        train.append(essay)
    else:
        valid.append(essay)

In [130]:
len(train), len(valid)

(312, 88)

In [131]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [132]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain_transformers.py \
    --output_dir=essays_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

## Check our model

In [140]:
## Select topic
for idx in range(len(valid)):
  if "образует его общество" in valid[idx]:
    break

In [141]:
valid[idx]

'<s>Тема: «Создает человека природа, но развивает и образует его общество». (В.Т. Белинский)\nСочинение: Эссе №1Человек это высшая ступень развития живых организмов на земле, субъект общественно-исторической деятельности и культуры, но важнейшей его характеристикой является биосоциальная сущность.Белинский В.Г. в своем выражении очень точно и ёмко охарактеризован двойственную природу человека. Во-первых, человек есть порождение природы, является цепью эволюции, таким же организмом, как и всё, что нас окружает. По биологическим признакам человек ничем не отличается от животного. Во-вторых, он есть порождение общества. Этот момент более сложен. Ясно только то, что благодаря общественному развитию человек стал человеком.Человек без общества ничто, недаром в древности изгнание из общества являлось самим страшным наказанием. Очень много в современном мире примеров синдрома«маугли», когда воспитание ребенок получил от животного и поэтому ведет себя как он, а не как человек, что говорит лишь 

In [138]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=essays_model \
    --k=5 \
    --p=0.95 \
    --length=500

2020-10-28 15:45:33.604619: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   Model name 'essays_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'essays_model' is a path, a model identifier, or url to a directory containing tokenizer files.
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   Didn't find file essays_model/added_tokens.json. We won't load it.
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/vocab.json
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/merges.txt
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file None
10/28/2020 15:45:35 - INFO - transformers.tokenization_utils -   loading file essays_model/special_tokens_map.json
10/28/2020 15:45:35 - INFO - tra